# Replication study

Replicate the DeepAR results for:

- Rangapuram, Syama Sundar, et al. "Deep state space models for time series forecasting." Advances in Neural Information Processing Systems. 2018.

## Data

Among the data there are two publicly available datasets, namely `electricity` and `traffic`

- `electricity` - Hourly time series of electricity consumption of 370 customers
- `traffic` - Hourly occupancy rates (between 0 and 1) of 963 car lanes of San Francisco bay area freeways

Both datasets exhibit hourly and daily seasonal patterns.

### Benchmark Methods 

- `auto.arima` - automatic ARIMA framework from R *forecast* package
- `ets` - automatic expoential smoothing, also from R *forecast* package

Benchmark methods are compared to `DeepAR` (this notebook) and `DeepState` (next notebook). For the paper, the built-in algorithm of the AWS SageMaker is used. I will see if the `gluonts.model.deepar` implementation is comparable.  

### RNN-based methods 

Inputs for `DeepAR`and `DeepState`: 

- Independent feature representing the category (i.e. the index)
- time-based features like hour-of-the-day, day-of-the-week, day-of-the-month
- `DeepState` - Size of state space model directly depends on the granularity of the time series which determines the number of seasons. 
 - For hourly data - hour-of-the day (24 seasons), day-of-the-week (7 seasons) models and hence latent dimension is 31
- Train each method on all time series of these datasets but vary the size of the training range $T_i \in {14, 21, 28}$ days.
- Evaluate on the next $\tau = 7$ days using *p50* and *p90* quantile loss.

#### Imports

In [29]:
# standard imports
import numpy as np
# import pandas as pd

# json
# import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

# seeds
mx.random.seed(42)
# np.random.seed(42)


In [2]:
dataset = get_dataset("electricity")

# Inspect data
print(len(list(dataset.train)))
print(len(list(dataset.test)))

entry = list(dataset.test)[0]

print("entry:\n", entry)
print("metadata: \n", dataset.metadata)
print("prediction_length:n", dataset.metadata.prediction_length)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\electricity.


321
2247
entry:
 {'start': Timestamp('2012-01-01 00:00:00', freq='H'), 'target': array([14., 18., 21., ...,  8.,  8.,  6.], dtype=float32), 'feat_static_cat': array([0]), 'source': SourceContext(source=Span(path=WindowsPath('C:/Users/TM/.mxnet/gluon-ts/datasets/electricity/test/data.json'), line=1), row=1)}
metadata: 
 MetaData freq='1H' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='321'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=24
prediction_length:n 24


In [3]:
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,      # default: 100
        batch_size=32,      # default: 32
        num_batches_per_epoch=50,      #default: 50
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 19863
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:26<00:00,  1.90it/s, avg_epoch_loss=6.69]
INFO:root:Epoch[0] Elapsed time 26.269 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.689123
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:18<00:00,  2.67it/s, avg_epoch_loss=5.88]
INFO:root:Epoch[1] Elapsed time 18.730 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=5.882805
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:15<00:00,  3.22it/s, avg_epoch_loss=5.71]
INFO:root:Epoch[2] Elapsed time 15.520 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=5.711921
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:19<00:00,  2.57it/s, avg_epoch_loss=5.58]
INFO:root:Epoch[3] Elapsed time 19.463 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=5.583702
INFO:root:Epoch[4] Learning rate is 0.

INFO:root:Epoch[36] Elapsed time 16.136 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=5.173382
INFO:root:Epoch[37] Learning rate is 0.0005
100%|██████████| 50/50 [00:19<00:00,  2.61it/s, avg_epoch_loss=5.11]
INFO:root:Epoch[37] Elapsed time 19.177 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=5.107152
INFO:root:Epoch[38] Learning rate is 0.0005
100%|██████████| 50/50 [00:16<00:00,  2.98it/s, avg_epoch_loss=5.15]
INFO:root:Epoch[38] Elapsed time 16.798 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=5.146046
INFO:root:Epoch[39] Learning rate is 0.0005
100%|██████████| 50/50 [00:15<00:00,  3.13it/s, avg_epoch_loss=5.15]
INFO:root:Epoch[39] Elapsed time 15.979 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=5.151310
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 50/50 [00:15<00:00,  3.25it/s, avg_epoch_loss=5.16]
INFO:root:Epoch[40] Elapsed time 15.386 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=5.155500
INFO:roo

100%|██████████| 50/50 [00:13<00:00,  3.81it/s, avg_epoch_loss=5.17]
INFO:root:Epoch[73] Elapsed time 13.120 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.174998
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.59it/s, avg_epoch_loss=5.1] 
INFO:root:Epoch[74] Elapsed time 13.939 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.101086
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.74it/s, avg_epoch_loss=5.03]
INFO:root:Epoch[75] Elapsed time 13.387 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.031799
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 50/50 [00:16<00:00,  3.10it/s, avg_epoch_loss=5.1] 
INFO:root:Epoch[76] Elapsed time 16.155 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.096562
INFO:root:Epoch[77] Learning rate is 0.00025
100%|██████████| 50/50 [00:15<00:00,  3.21it/s, avg_epoch_loss=5.08]
INFO:root:Epoch[77] Elapsed time 15.560 sec

{'Coverage[0.1]': 0.08436645686311892,
 'Coverage[0.2]': 0.15595397037319628,
 'Coverage[0.3]': 0.23161040116981485,
 'Coverage[0.4]': 0.3067582173056144,
 'Coverage[0.5]': 0.3899167143492924,
 'Coverage[0.6]': 0.47510967003623866,
 'Coverage[0.7]': 0.5765782948693492,
 'Coverage[0.8]': 0.6845317566278868,
 'Coverage[0.9]': 0.8027210884353791,
 'MAE_Coverage': 0.08805038110778993,
 'MASE': 0.8330967249588235,
 'MSE': 5258773.763251051,
 'MSIS': 10.944287886928509,
 'ND': 0.07437302412764038,
 'NRMSE': 0.7106490787782086,
 'QuantileLoss[0.1]': 1413345.8631078955,
 'QuantileLoss[0.2]': 2313039.9757389817,
 'QuantileLoss[0.3]': 2971822.365715612,
 'QuantileLoss[0.4]': 3449134.6758742547,
 'QuantileLoss[0.5]': 3774883.954206478,
 'QuantileLoss[0.6]': 3937504.8743052958,
 'QuantileLoss[0.7]': 3899732.362636425,
 'QuantileLoss[0.8]': 3596312.208871216,
 'QuantileLoss[0.9]': 2882575.0383782685,
 'RMSE': 2293.2016403384705,
 'abs_error': 3774883.906194687,
 'abs_target_mean': 3226.911437472181

In [4]:
dataset = get_dataset("traffic")

# Inspect data
print(len(list(dataset.train)))
print(len(list(dataset.test)))

entry = list(dataset.test)[0]

print("entry:\n", entry)
print("metadata: \n", dataset.metadata)
print("prediction_length:n", dataset.metadata.prediction_length)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\traffic.


862
6034
entry:
 {'start': Timestamp('2015-01-01 00:00:00', freq='H'), 'target': array([0.0048, 0.0072, 0.004 , ..., 0.0467, 0.0412, 0.0386], dtype=float32), 'feat_static_cat': array([0]), 'source': SourceContext(source=Span(path=WindowsPath('C:/Users/TM/.mxnet/gluon-ts/datasets/traffic/test/data.json'), line=1), row=1)}
metadata: 
 MetaData freq='H' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='862'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=24
prediction_length:n 24


In [44]:
dataset = get_dataset("traffic")
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,      # default: 100
        batch_size=32,      # default: 32
        num_batches_per_epoch=50,      #default: 50
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\traffic.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 30683
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  3.90it/s, avg_epoch_loss=-2.35]
INFO:root:Epoch[0] Elapsed time 12.825 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-2.345696
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.39it/s, avg_epoch_loss=-3.15]
INFO:root:Epoch[1] Elapsed time 11.391 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=-3.149448
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  3.87it/s, avg_epoch_loss=-3.35]
INFO:root:Epoch[2] Elapsed time 12.942 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-3.354219
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.51it/s, avg_epoch_loss=-3.4] 
INFO:root:Epoch[3] Elapsed time 11.097 se

100%|██████████| 50/50 [00:09<00:00,  5.27it/s, avg_epoch_loss=-3.9]
INFO:root:Epoch[36] Elapsed time 9.496 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=-3.895965
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.31it/s, avg_epoch_loss=-3.92]
INFO:root:Epoch[37] Elapsed time 9.418 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=-3.924807
INFO:root:Epoch[38] Learning rate is 0.001
  0%|          | 0/50 [00:09<?, ?it/s, avg_epoch_loss=-3.97]


KeyboardInterrupt: 

In [ ]:
dataset = get_dataset("traffic")
mx.random.seed(43)
np.random.seed(43)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,      # default: 100
        batch_size=32,      # default: 32
        num_batches_per_epoch=50,      #default: 50
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


In [ ]:
dataset = get_dataset("traffic")
mx.random.seed(44)
np.random.seed(44)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,      # default: 100
        batch_size=32,      # default: 32
        num_batches_per_epoch=50,      #default: 50
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


In [11]:
dataset = get_dataset("m4_hourly")

# Inspect data
print(len(list(dataset.train)))
print(len(list(dataset.test)))

entry = list(dataset.test)[16]

print("entry:\n", entry)
print("metadata: \n", dataset.metadata)
print("prediction_length:n", dataset.metadata.prediction_length)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_hourly.


414
414
entry:
 {'start': Timestamp('1750-01-01 00:00:00', freq='H'), 'target': array([52694., 50487., 48905., 47791., 44692., 40769., 37456., 35119.,
       33638., 32798., 32805., 34121., 35894., 37403., 39964., 43192.,
       46593., 48829., 50231., 51645., 52622., 53774., 53665., 52925.,
       51756., 50171., 48655., 47667., 44629., 40754., 37547., 35199.,
       33637., 32759., 32536., 33527., 35196., 36637., 39221., 42398.,
       45675., 48667., 51260., 53719., 55525., 56836., 57505., 56807.,
       55044., 52776., 50986., 49845., 46817., 43307., 40173., 37954.,
       36353., 35346., 34981., 35402., 35919., 36447., 38820., 42009.,
       45201., 48116., 50501., 52232., 53443., 54180., 54510., 54028.,
       52732., 50673., 49028., 47940., 45385., 42470., 39566., 37379.,
       35757., 34698., 34057., 33939., 33910., 34456., 37333., 41075.,
       45018., 48470., 51173., 52980., 54226., 54755., 54722., 54308.,
       52974., 50742., 48474., 46782., 44767., 42872., 40437., 38209

## Data: `m4_hourly`

In [24]:
dataset = get_dataset("m4_hourly")

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_hourly.


#### Run 1

In [34]:
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,     
#         batch_size=32,      
        num_batches_per_epoch=50,     
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
#         context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=48,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.99it/s, avg_epoch_loss=5.28]
INFO:root:Epoch[0] Elapsed time 10.020 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.280023
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.45it/s, avg_epoch_loss=4.52]
INFO:root:Epoch[1] Elapsed time 9.186 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.521267
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:08<00:00,  5.56it/s, avg_epoch_loss=4.16]
INFO:root:Epoch[2] Elapsed time 8.999 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.156798
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:08<00:00,  5.85it/s, avg_epoch_loss=3.66]
INFO:root:Epoch[3] Elapsed time 8.559 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=3.663846
INFO:root:Epoch[4] Learning rate is 0.001

INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.64it/s, avg_epoch_loss=3.19]
INFO:root:Epoch[37] Elapsed time 10.791 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=3.188947
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.75it/s, avg_epoch_loss=2.91]
INFO:root:Epoch[38] Elapsed time 10.537 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=2.912309
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.87it/s, avg_epoch_loss=3.09]
INFO:root:Epoch[39] Elapsed time 10.270 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=3.094895
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.19it/s, avg_epoch_loss=3.15]
INFO:root:Epoch[40] Elapsed time 9.632 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=3.147061
INFO:root:Epoch[41] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.98it/s, avg_epoch_loss=3.2]
INFO:root:

INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.15it/s, avg_epoch_loss=2.88]
INFO:root:Epoch[74] Elapsed time 9.724 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.881696
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:10<00:00,  4.86it/s, avg_epoch_loss=2.68]
INFO:root:Epoch[75] Elapsed time 10.297 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.684727
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:16<00:00,  3.03it/s, avg_epoch_loss=2.82]
INFO:root:Epoch[76] Elapsed time 16.515 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.817950
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.08it/s, avg_epoch_loss=2.83]
INFO:root:Epoch[77] Elapsed time 9.841 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=2.833692
INFO:root:Epoch[78] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.37it/s, avg_epoch_loss=2.59]
INFO:

{'Coverage[0.1]': 0.2193035426731077,
 'Coverage[0.2]': 0.3049013687600642,
 'Coverage[0.3]': 0.3820954106280194,
 'Coverage[0.4]': 0.4523953301127216,
 'Coverage[0.5]': 0.5211855877616749,
 'Coverage[0.6]': 0.580968196457327,
 'Coverage[0.7]': 0.6506139291465377,
 'Coverage[0.8]': 0.7207628824476646,
 'Coverage[0.9]': 0.8006239935587769,
 'MAE_Coverage': 0.06965691536947573,
 'MASE': 1.339494444687416,
 'MSE': 4183684.922365106,
 'MSIS': 21.736033959710518,
 'ND': 0.048259866751103414,
 'NRMSE': 0.27924307201782844,
 'QuantileLoss[0.1]': 4014071.236398793,
 'QuantileLoss[0.2]': 5449269.579226495,
 'QuantileLoss[0.3]': 6326800.884770488,
 'QuantileLoss[0.4]': 6812774.4857089985,
 'QuantileLoss[0.5]': 7024651.359565735,
 'QuantileLoss[0.6]': 6920977.141727828,
 'QuantileLoss[0.7]': 6542560.461486817,
 'QuantileLoss[0.8]': 5829813.100245284,
 'QuantileLoss[0.9]': 4646873.226919174,
 'RMSE': 2045.4058087247886,
 'abs_error': 7024651.361759186,
 'abs_target_mean': 7324.822041043147,
 'abs_

#### Run 2

In [39]:
mx.random.seed(43)
np.random.seed(43)

if __name__ == "__main__":
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,      
#         batch_size=32,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
#         context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=48,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:17<00:00,  2.79it/s, avg_epoch_loss=5.52]
INFO:root:Epoch[0] Elapsed time 17.934 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.516625
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  4.06it/s, avg_epoch_loss=4.58]
INFO:root:Epoch[1] Elapsed time 12.333 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.577677
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.71it/s, avg_epoch_loss=4.08]
INFO:root:Epoch[2] Elapsed time 10.621 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.082698
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:15<00:00,  3.18it/s, avg_epoch_loss=3.77]
INFO:root:Epoch[3] Elapsed time 15.711 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=3.771905
INFO:root:Epoch[4] Learning rate is 0.

INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.67it/s, avg_epoch_loss=3.03]
INFO:root:Epoch[37] Elapsed time 10.716 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=3.026024
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.67it/s, avg_epoch_loss=3.27]
INFO:root:Epoch[38] Elapsed time 10.724 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=3.269804
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.85it/s, avg_epoch_loss=3.24]
INFO:root:Epoch[39] Elapsed time 10.311 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=3.235729
INFO:root:Loading parameters from best epoch (29)
INFO:root:Epoch[40] Learning rate is 0.0005
100%|██████████| 50/50 [00:10<00:00,  4.78it/s, avg_epoch_loss=3.32]
INFO:root:Epoch[40] Elapsed time 10.476 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=3.317643
INFO:root:Epoch[41] Learning rate is 0.0005
100%|██████████| 50/50 [0

100%|██████████| 50/50 [00:11<00:00,  4.24it/s, avg_epoch_loss=2.9] 
INFO:root:Epoch[73] Elapsed time 11.806 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=2.904866
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 50/50 [00:11<00:00,  4.30it/s, avg_epoch_loss=2.93]
INFO:root:Epoch[74] Elapsed time 11.659 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.926818
INFO:root:Epoch[75] Learning rate is 0.000125
100%|██████████| 50/50 [00:11<00:00,  4.37it/s, avg_epoch_loss=2.9] 
INFO:root:Epoch[75] Elapsed time 11.458 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.903587
INFO:root:Epoch[76] Learning rate is 0.000125
100%|██████████| 50/50 [00:12<00:00,  4.07it/s, avg_epoch_loss=2.98]
INFO:root:Epoch[76] Elapsed time 12.293 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.979229
INFO:root:Loading parameters from best epoch (66)
INFO:root:Epoch[77] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:11<00:00,  4.29it/s, avg_epoch_

{'Coverage[0.1]': 0.19439412238325268,
 'Coverage[0.2]': 0.26494565217391286,
 'Coverage[0.3]': 0.32981078904991945,
 'Coverage[0.4]': 0.3883856682769727,
 'Coverage[0.5]': 0.4466586151368762,
 'Coverage[0.6]': 0.502969001610306,
 'Coverage[0.7]': 0.5683373590982284,
 'Coverage[0.8]': 0.6360708534621576,
 'Coverage[0.9]': 0.7220712560386464,
 'MAE_Coverage': 0.09162864555376642,
 'MASE': 1.4937566648691005,
 'MSE': 7387000.336408353,
 'MSIS': 27.869067055412792,
 'ND': 0.06411661686083005,
 'NRMSE': 0.3710538837040973,
 'QuantileLoss[0.1]': 8834856.205185318,
 'QuantileLoss[0.2]': 9985400.288142204,
 'QuantileLoss[0.3]': 10237161.856968973,
 'QuantileLoss[0.4]': 9972057.491693115,
 'QuantileLoss[0.5]': 9332741.979476929,
 'QuantileLoss[0.6]': 8346155.960227966,
 'QuantileLoss[0.7]': 7077066.434729195,
 'QuantileLoss[0.8]': 5492653.74414978,
 'QuantileLoss[0.9]': 3493867.861057472,
 'RMSE': 2717.9036657704323,
 'abs_error': 9332741.888113976,
 'abs_target_mean': 7324.822041043147,
 'abs

#### Run 3

In [40]:
mx.random.seed(44)
np.random.seed(44)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,     
#         batch_size=32,      
        num_batches_per_epoch=50,     
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
#         context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=48,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  3.98it/s, avg_epoch_loss=5.11]
INFO:root:Epoch[0] Elapsed time 12.571 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.111396
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.25it/s, avg_epoch_loss=4.26]
INFO:root:Epoch[1] Elapsed time 11.775 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.256519
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.27it/s, avg_epoch_loss=4.27]
INFO:root:Epoch[2] Elapsed time 11.713 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.270225
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.31it/s, avg_epoch_loss=4.05]
INFO:root:Epoch[3] Elapsed time 11.611 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=4.050069
INFO:root:Epoch[4] Learning rate is 0.

INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=3.073500
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.40it/s, avg_epoch_loss=3.19]
INFO:root:Epoch[37] Elapsed time 11.388 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=3.185430
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.36it/s, avg_epoch_loss=3.16]
INFO:root:Epoch[38] Elapsed time 11.482 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=3.156585
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.46it/s, avg_epoch_loss=3.62]
INFO:root:Epoch[39] Elapsed time 11.227 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=3.620459
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  4.05it/s, avg_epoch_loss=3.24]
INFO:root:Epoch[40] Elapsed time 12.355 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=3.242242
INFO:root:Epoch[41] Learning rate is 0.001
100%|██████████| 

INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=2.948231
INFO:root:Loading parameters from best epoch (63)
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.54it/s, avg_epoch_loss=2.92]
INFO:root:Epoch[74] Elapsed time 9.038 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.916431
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.32it/s, avg_epoch_loss=2.89]
INFO:root:Epoch[75] Elapsed time 9.399 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.886410
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.53it/s, avg_epoch_loss=2.45]
INFO:root:Epoch[76] Elapsed time 9.051 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.453312
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.40it/s, avg_epoch_loss=2.95]
INFO:root:Epoch[77] Elapsed time 9.266 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=2.951038
INFO:root:

{'Coverage[0.1]': 0.2977053140096616,
 'Coverage[0.2]': 0.3756541867954912,
 'Coverage[0.3]': 0.4387580515297907,
 'Coverage[0.4]': 0.493961352657005,
 'Coverage[0.5]': 0.5434782608695653,
 'Coverage[0.6]': 0.5849939613526569,
 'Coverage[0.7]': 0.6347624798711753,
 'Coverage[0.8]': 0.6896135265700485,
 'Coverage[0.9]': 0.7486413043478265,
 'MAE_Coverage': 0.11017176596886741,
 'MASE': 2.162661147215832,
 'MSE': 51197135.181636654,
 'MSIS': 47.482280539307204,
 'ND': 0.13483371425025703,
 'NRMSE': 0.9768452120525684,
 'QuantileLoss[0.1]': 5642416.087568759,
 'QuantileLoss[0.2]': 9797066.959729765,
 'QuantileLoss[0.3]': 13430605.929391384,
 'QuantileLoss[0.4]': 16704168.73035393,
 'QuantileLoss[0.5]': 19626242.385269165,
 'QuantileLoss[0.6]': 22246088.176530074,
 'QuantileLoss[0.7]': 24401930.14681969,
 'QuantileLoss[0.8]': 25980818.0787632,
 'QuantileLoss[0.9]': 26527825.87313557,
 'RMSE': 7155.217339930119,
 'abs_error': 19626242.22118187,
 'abs_target_mean': 7324.822041043147,
 'abs_t

### m4_weekly

#### Run 1

In [41]:
dataset = get_dataset("m4_weekly")

mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,     
#         batch_size=32,      
        num_batches_per_epoch=50,     
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
#         context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_weekly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 20623
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 12.69it/s, avg_epoch_loss=7.69]
INFO:root:Epoch[0] Elapsed time 3.951 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.689772
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 16.02it/s, avg_epoch_loss=7.04]
INFO:root:Epoch[1] Elapsed time 3.125 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.041310
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 15.74it/s, avg_epoch_loss=6.98]
INFO:root:Epoch[2] Elapsed time 3.182 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.981438
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 16.10it/s, avg_epoch_loss=6.88]
INFO:root:Epoch[3] Elapsed time 3.111 seconds
INF

INFO:root:Epoch[36] Elapsed time 3.182 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.048322
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 15.57it/s, avg_epoch_loss=6.03]
INFO:root:Epoch[37] Elapsed time 3.218 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.027876
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 14.45it/s, avg_epoch_loss=6.09]
INFO:root:Epoch[38] Elapsed time 3.465 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.085090
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 15.38it/s, avg_epoch_loss=6.17]
INFO:root:Epoch[39] Elapsed time 3.258 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.166363
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 16.12it/s, avg_epoch_loss=6.06]
INFO:root:Epoch[40] Elapsed time 3.107 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=6.061896
INFO:root:Epoch[4

INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.867869
INFO:root:Epoch[74] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 15.72it/s, avg_epoch_loss=5.96]
INFO:root:Epoch[74] Elapsed time 3.188 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.959474
INFO:root:Epoch[75] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 15.85it/s, avg_epoch_loss=5.99]
INFO:root:Epoch[75] Elapsed time 3.159 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.990572
INFO:root:Epoch[76] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 16.19it/s, avg_epoch_loss=6]
INFO:root:Epoch[76] Elapsed time 3.094 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=6.001748
INFO:root:Epoch[77] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 16.30it/s, avg_epoch_loss=5.89]
INFO:root:Epoch[77] Elapsed time 3.074 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=5.890187
INFO:root:Epoch[78] Learning rate is 0.001
100%|██████████| 50/50 [

{'Coverage[0.1]': 0.06985215341761297,
 'Coverage[0.2]': 0.12877651596314568,
 'Coverage[0.3]': 0.2224126848082286,
 'Coverage[0.4]': 0.3173344761088497,
 'Coverage[0.5]': 0.41268480822798365,
 'Coverage[0.6]': 0.4973216198842942,
 'Coverage[0.7]': 0.6035997428755092,
 'Coverage[0.8]': 0.7250910649239348,
 'Coverage[0.9]': 0.8510820655667456,
 'MAE_Coverage': 0.07464942980263284,
 'MASE': 2.630469230158148,
 'MSE': 463435.92852001445,
 'MSIS': 25.39900077769161,
 'ND': 0.06683447383259322,
 'NRMSE': 0.12402414904712314,
 'QuantileLoss[0.1]': 774795.7307128906,
 'QuantileLoss[0.2]': 1201310.104397583,
 'QuantileLoss[0.3]': 1484219.4792755125,
 'QuantileLoss[0.4]': 1657422.5794128417,
 'QuantileLoss[0.5]': 1712091.3278884888,
 'QuantileLoss[0.6]': 1680754.8343353271,
 'QuantileLoss[0.7]': 1497688.4356170653,
 'QuantileLoss[0.8]': 1213732.2112640382,
 'QuantileLoss[0.9]': 794988.9272232056,
 'RMSE': 680.761286002674,
 'abs_error': 1712091.331123352,
 'abs_target_mean': 5488.941397566194,


#### Run 2

In [42]:
dataset = get_dataset("m4_weekly")

mx.random.seed(43)
np.random.seed(43)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,     
#         batch_size=32,      
        num_batches_per_epoch=50,     
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
#         context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_weekly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 20623
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 13.23it/s, avg_epoch_loss=7.85]
INFO:root:Epoch[0] Elapsed time 3.786 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.851841
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 15.44it/s, avg_epoch_loss=7.13]
INFO:root:Epoch[1] Elapsed time 3.245 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.132012
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 15.69it/s, avg_epoch_loss=6.81]
INFO:root:Epoch[2] Elapsed time 3.199 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.813863
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 13.87it/s, avg_epoch_loss=6.83]
INFO:root:Epoch[3] Elapsed time 3.610 seconds
INF

INFO:root:Epoch[36] Elapsed time 3.268 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.010220
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 15.61it/s, avg_epoch_loss=6.04]
INFO:root:Epoch[37] Elapsed time 3.209 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.041358
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 15.69it/s, avg_epoch_loss=5.95]
INFO:root:Epoch[38] Elapsed time 3.192 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=5.951913
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 16.14it/s, avg_epoch_loss=6.09]
INFO:root:Epoch[39] Elapsed time 3.103 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.090437
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 14.76it/s, avg_epoch_loss=6.05]
INFO:root:Epoch[40] Elapsed time 3.393 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=6.050449
INFO:root:Epoch[4

INFO:root:Epoch[73] Elapsed time 3.396 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.865908
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:03<00:00, 15.98it/s, avg_epoch_loss=5.81]
INFO:root:Epoch[74] Elapsed time 3.134 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.811031
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:03<00:00, 15.60it/s, avg_epoch_loss=5.94]
INFO:root:Epoch[75] Elapsed time 3.213 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.938394
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:03<00:00, 16.50it/s, avg_epoch_loss=5.91]
INFO:root:Epoch[76] Elapsed time 3.036 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.913470
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:03<00:00, 15.81it/s, avg_epoch_loss=5.9]
INFO:root:Epoch[77] Elapsed time 3.166 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=5.897485
INFO:root:Epoc

{'Coverage[0.1]': 0.07263766873794723,
 'Coverage[0.2]': 0.13177630169273652,
 'Coverage[0.3]': 0.2095564602528395,
 'Coverage[0.4]': 0.2989072209127923,
 'Coverage[0.5]': 0.38675808870794964,
 'Coverage[0.6]': 0.4776087422326978,
 'Coverage[0.7]': 0.5961002785515319,
 'Coverage[0.8]': 0.7302335547460895,
 'Coverage[0.9]': 0.8697235911720593,
 'MAE_Coverage': 0.08074423255481737,
 'MASE': 2.5730702456645194,
 'MSE': 444685.53919288766,
 'MSIS': 26.29675246354033,
 'ND': 0.06431914565412042,
 'NRMSE': 0.1214892663106473,
 'QuantileLoss[0.1]': 791005.5717193603,
 'QuantileLoss[0.2]': 1222920.58303833,
 'QuantileLoss[0.3]': 1487896.1904998776,
 'QuantileLoss[0.4]': 1628356.8238769532,
 'QuantileLoss[0.5]': 1647656.4454345703,
 'QuantileLoss[0.6]': 1572611.8897827147,
 'QuantileLoss[0.7]': 1377449.5158813477,
 'QuantileLoss[0.8]': 1101050.254586792,
 'QuantileLoss[0.9]': 723240.0602020263,
 'RMSE': 666.8474632124559,
 'abs_error': 1647656.447113037,
 'abs_target_mean': 5488.941397566194,
 

#### Run 3

In [43]:
dataset = get_dataset("m4_weekly")

mx.random.seed(44)
np.random.seed(44)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,     
#         batch_size=32,      
        num_batches_per_epoch=50,     
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
#         context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_weekly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 20623
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.73it/s, avg_epoch_loss=7.77]
INFO:root:Epoch[0] Elapsed time 4.693 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.774016
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 12.35it/s, avg_epoch_loss=7.08]
INFO:root:Epoch[1] Elapsed time 4.055 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.077434
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 12.16it/s, avg_epoch_loss=6.87]
INFO:root:Epoch[2] Elapsed time 4.120 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.869542
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 12.92it/s, avg_epoch_loss=6.82]
INFO:root:Epoch[3] Elapsed time 3.878 seconds
INF

INFO:root:Epoch[36] Elapsed time 4.231 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=6.127215
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.89it/s, avg_epoch_loss=6.05]
INFO:root:Epoch[37] Elapsed time 4.607 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=6.052713
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.68it/s, avg_epoch_loss=6]
INFO:root:Epoch[38] Elapsed time 4.698 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=6.003522
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 14.68it/s, avg_epoch_loss=6.01]
INFO:root:Epoch[39] Elapsed time 3.415 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=6.010694
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 14.19it/s, avg_epoch_loss=5.94]
INFO:root:Epoch[40] Elapsed time 3.571 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=5.942229
INFO:root:Epoch[41] 

INFO:root:Epoch[73] Elapsed time 4.020 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.850110
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 12.02it/s, avg_epoch_loss=5.9]
INFO:root:Epoch[74] Elapsed time 4.174 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.902186
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:03<00:00, 12.89it/s, avg_epoch_loss=5.87]
INFO:root:Epoch[75] Elapsed time 3.884 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.873208
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 12.42it/s, avg_epoch_loss=5.91]
INFO:root:Epoch[76] Elapsed time 4.034 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.905972
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:04<00:00, 12.28it/s, avg_epoch_loss=5.9]
INFO:root:Epoch[77] Elapsed time 4.081 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=5.897753
INFO:root:Epoch

{'Coverage[0.1]': 0.0947075208913651,
 'Coverage[0.2]': 0.17870152131990596,
 'Coverage[0.3]': 0.27126633811870626,
 'Coverage[0.4]': 0.37025926719520014,
 'Coverage[0.5]': 0.46325262481251345,
 'Coverage[0.6]': 0.5483179772873364,
 'Coverage[0.7]': 0.650739232911935,
 'Coverage[0.8]': 0.7572316263124064,
 'Coverage[0.9]': 0.8667238054424693,
 'MAE_Coverage': 0.033200009523129105,
 'MASE': 2.543950655249818,
 'MSE': 430494.41801636695,
 'MSIS': 25.79170701329904,
 'ND': 0.06339690394106345,
 'NRMSE': 0.11953502265844558,
 'QuantileLoss[0.1]': 749831.4343658448,
 'QuantileLoss[0.2]': 1169120.443902588,
 'QuantileLoss[0.3]': 1434002.9883483886,
 'QuantileLoss[0.4]': 1577377.5205932618,
 'QuantileLoss[0.5]': 1624031.485069275,
 'QuantileLoss[0.6]': 1579563.2512084963,
 'QuantileLoss[0.7]': 1420682.6877319338,
 'QuantileLoss[0.8]': 1164805.247817993,
 'QuantileLoss[0.9]': 765389.2768310546,
 'RMSE': 656.1207343289549,
 'abs_error': 1624031.4830551147,
 'abs_target_mean': 5488.941397566194,

In [6]:
# import rpy2
# print(rpy2.__version__)

# # importing packages
# from rpy2.robjects.packages import importr
# import rpy2.robjects.packages as rpackages

# base=importr("base")
# utils=importr("utils")

# utils = rpackages.importr("utils")
# utils.chooseCRANmirror(ind=1)

# utils.install_packages("forecast")
# utils.install_packages("nnfor")


In [7]:
print("Done!")

Done!
